# ID3 Algorithm

In [123]:
# Code by Bhavy Kharbanda
# Sap Id: 500082531

In [124]:
# References:
# https://towardsdatascience.com/decision-trees-for-classification-id3-algorithm-explained-89df76e72df1
# https://www.youtube.com/watch?v=coOTEc-0OGw

In [125]:
# ID3 stands for Iterative Dichotomiser 3 and is named such because the algorithm iteratively (repeatedly) dichotomizes(divides) features into two or more groups at each step. ID3 uses a top-down greedy approach to build a decision tree. In simple words, the top-down approach means that we start building the tree from the top and the greedy approach means that at each iteration we select the best feature at the present moment to create a node.

In [126]:
# How the best feature is selected on each node?
# Information Gain calculates the reduction in the entropy and measures how well a given feature separates or classifies the target classes. The feature with the highest Information Gain is selected as the best one.

In [127]:
# Entropy is Calculated as: 
# Entropy(S) = - ∑ pᵢ * log₂(pᵢ) ; i = 1 to n

# n is the total number of classes in the target column 
# pᵢ is the probability of class ‘i’ or the ratio of “number of rows with class i in the target column” to the “total number of rows” in the dataset.

# Information Gain for a feature column A is calculated as:
# IG(S, A) = Entropy(S) - ∑((|Sᵥ| / |S|) * Entropy(Sᵥ))

In [128]:
# >>>> ID3 Steps
# 1. Calculate the Information Gain of each feature.
# 2. Considering that all rows don’t belong to the same class, split the dataset S into subsets using the feature for which the Information Gain is maximum.
# 3. Make a decision tree node using the feature with the maximum Information gain.
# 4. If all rows belong to the same class, make the current node as a leaf node with the class as its label.
# 5. Repeat for the remaining features until we run out of all features, or the decision tree has all leaf nodes.

In [129]:
# importing libraries
import math
import pandas as pd
import numpy as np


# Testing on the Dataset

In [130]:
data = pd.read_csv("Datasets\Weather_id3.csv")
data.head()

,Outlook,Temperature,Humidity,Wind,Answer
0,sunny,hot,high,weak,no
1,sunny,hot,high,strong,no
2,overcast,hot,high,weak,yes
3,rain,mild,high,weak,yes
4,rain,cool,normal,weak,yes


In [131]:
features = [feat for feat in data]
features.remove("Answer")

In [132]:
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

In [133]:
def entropy(examples):
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        if row["Answer"] == "yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        return 0.0
    else:
        p = pos / (pos + neg)
        n = neg / (pos + neg)
        return -(p * math.log(p, 2) + n * math.log(n, 2))

In [156]:
def info_gain(examples, attr):
    uniq = np.unique(examples[attr])
    
    gain = entropy(examples)
    
    for u in uniq:
        subdata = examples[examples[attr] == u]
        sub_e = entropy(subdata)
        gain -= (float(len(subdata)) / float(len(examples))) * sub_e
    return gain

In [135]:
def ID3(examples, attrs):
    root = Node()

    max_gain = 0
    max_feat = ""
    for feature in attrs:
       
        gain = info_gain(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value = max_feat
 
    uniq = np.unique(examples[max_feat])

    for u in uniq:
     
        subdata = examples[examples[max_feat] == u]
     
        if entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["Answer"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)

    return root

In [136]:
def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

In [137]:
def classify(root: Node, new):
    for child in root.children:
        if child.value == new[root.value]:
            if child.isLeaf:
                print ("Predicted Label for new example", new," is:", child.pred)
                exit
            else:
                classify (child.children[0], new)

In [138]:
root = ID3(data, features)
print("Decision Tree is:")
printTree(root)
print ("------------------")


Decision Tree is:
Outlook
	overcast ->  ['yes']

	rain
		Wind
			strong ->  ['no']

			weak ->  ['yes']

	sunny
		Humidity
			high ->  ['no']

			normal ->  ['yes']

------------------


### Testing on New Features

In [139]:
new = {"Outlook":"sunny", "Temperature":"hot", "Humidity":"normal", "Wind":"strong"}
classify (root, new)

Predicted Label for new example {'Outlook': 'sunny', 'Temperature': 'hot', 'Humidity': 'normal', 'Wind': 'strong'}  is: ['yes']


In [140]:
new = {"Outlook":"rain", "Temperature":"cool", "Humidity":"normal", "Wind":"strong"}
classify (root, new)

Predicted Label for new example {'Outlook': 'rain', 'Temperature': 'cool', 'Humidity': 'normal', 'Wind': 'strong'}  is: ['no']


In [141]:
new = {"Outlook":"sunny", "Temperature":"mild", "Humidity":"normal", "Wind":"strong"}
classify (root, new)

Predicted Label for new example {'Outlook': 'sunny', 'Temperature': 'mild', 'Humidity': 'normal', 'Wind': 'strong'}  is: ['yes']


# Testing on 2nd Dataset

In [142]:
data = pd.read_csv("Datasets\diabetes_id3.csv")
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,Yes
1,1,85,66,29,0,26.6,0.351,31,No
2,8,183,64,0,0,23.3,0.672,32,Yes
3,1,89,66,23,94,28.1,0.167,21,No
4,0,137,40,35,168,43.1,2.288,33,Yes


In [143]:
features = [feat for feat in data]
features.remove("Outcome")

In [144]:
class Node:
    def __init__(self):
        self.children = []
        self.value = ""
        self.isLeaf = False
        self.pred = ""

In [145]:
def entropy(examples):
    pos = 0.0
    neg = 0.0
    for _, row in examples.iterrows():
        if row["Outcome"] == "Yes":
            pos += 1
        else:
            neg += 1
    if pos == 0.0 or neg == 0.0:
        return 0.0
    else:
        p = pos / (pos + neg)
        n = neg / (pos + neg)
        return -(p * math.log(p, 2) + n * math.log(n, 2))

In [146]:
def info_gain(examples, attr):
    uniq = np.unique(examples[attr])

    gain = entropy(examples)

    for u in uniq:
        subdata = examples[examples[attr] == u]
      
        sub_e = entropy(subdata)
        gain -= (float(len(subdata)) / float(len(examples))) * sub_e
     
    return gain

In [147]:
def ID3(examples, attrs):
    root = Node()

    max_gain = 0
    max_feat = ""
    for feature in attrs:
     
        gain = info_gain(examples, feature)
        if gain > max_gain:
            max_gain = gain
            max_feat = feature
    root.value = max_feat

    uniq = np.unique(examples[max_feat])
 
    for u in uniq:
      
        subdata = examples[examples[max_feat] == u]
 
        if entropy(subdata) == 0.0:
            newNode = Node()
            newNode.isLeaf = True
            newNode.value = u
            newNode.pred = np.unique(subdata["Outcome"])
            root.children.append(newNode)
        else:
            dummyNode = Node()
            dummyNode.value = u
            new_attrs = attrs.copy()
            new_attrs.remove(max_feat)
            child = ID3(subdata, new_attrs)
            dummyNode.children.append(child)
            root.children.append(dummyNode)

    return root

In [148]:
def printTree(root: Node, depth=0):
    for i in range(depth):
        print("\t", end="")
    print(root.value, end="")
    if root.isLeaf:
        print(" -> ", root.pred)
    print()
    for child in root.children:
        printTree(child, depth + 1)

In [149]:
def classify(root: Node, new):
    for child in root.children:
        if child.value == new[root.value]:
            if child.isLeaf:
                print ("Predicted Label for new example", new," is:", child.pred)
                exit
            else:
                classify (child.children[0], new)

In [150]:
root = ID3(data, features)
print("Decision Tree is:")
printTree(root)
print ("------------------")


Decision Tree is:
DiabetesPedigreeFunction
	0.078 ->  ['No']

	0.084 ->  ['No']

	0.085 ->  ['No']

	0.088
		Pregnancies
			1 ->  ['Yes']

			2 ->  ['No']

	0.089 ->  ['No']

	0.092 ->  ['No']

	0.096 ->  ['No']

	0.1 ->  ['No']

	0.101 ->  ['No']

	0.102 ->  ['No']

	0.107 ->  ['No']

	0.108 ->  ['No']

	0.115 ->  ['No']

	0.118 ->  ['No']

	0.121
		Pregnancies
			3 ->  ['No']

			6 ->  ['Yes']

	0.122 ->  ['No']

	0.123 ->  ['No']

	0.126 ->  ['No']

	0.127 ->  ['Yes']

	0.128
		Pregnancies
			2 ->  ['No']

			7 ->  ['Yes']

	0.129 ->  ['Yes']

	0.13 ->  ['No']

	0.133 ->  ['No']

	0.134 ->  ['No']

	0.135 ->  ['Yes']

	0.136 ->  ['No']

	0.137
		Pregnancies
			8 ->  ['Yes']

			12 ->  ['No']

	0.138 ->  ['No']

	0.14 ->  ['No']

	0.141
		Pregnancies
			0 ->  ['Yes']

			2 ->  ['No']

			10 ->  ['Yes']

	0.142 ->  ['No']

	0.143 ->  ['No']

	0.144 ->  ['No']

	0.145 ->  ['No']

	0.147 ->  ['No']

	0.148
		Pregnancies
			2 ->  ['No']

			4 ->  ['No']

			9 ->  ['Yes']

	0.149 ->  ['No

### Testing on new datasets

In [152]:
new = {"Pregnancies":3, "Glucose":78, "BloodPressure":50, "SkinThickness":32, "Insulin":88, "BMI":31, "DiabetesPedigreeFunction":0.248, "Age":26}
classify (root, new)


Predicted Label for new example {'Pregnancies': 3, 'Glucose': 78, 'BloodPressure': 50, 'SkinThickness': 32, 'Insulin': 88, 'BMI': 31, 'DiabetesPedigreeFunction': 0.248, 'Age': 26}  is: ['Yes']


In [153]:
new = {"Pregnancies":1, "Glucose":89, "BloodPressure":66, "SkinThickness":23, "Insulin":94, "BMI":28.1, "DiabetesPedigreeFunction":0.167, "Age":21}
classify (root, new)

Predicted Label for new example {'Pregnancies': 1, 'Glucose': 89, 'BloodPressure': 66, 'SkinThickness': 23, 'Insulin': 94, 'BMI': 28.1, 'DiabetesPedigreeFunction': 0.167, 'Age': 21}  is: ['No']
